# Session 2 - Parameter Calibration

---
* Time: run all cells first time: ~ 4.5 minutes
* Time: run all cells second time: ~ 40 seconds
* Time: run and read step by step: ~ 20 minutes

---

Usually, the parameters we obtain from the characterization of the cell result is simulations that are off from cycling data. Therefore, before using a model, it is good practice to carry out a "Calibration" step, where we fit a small subset of the model parameters to experimental voltage profiles of the same cell.

In this session, we will calibrate the Xu 2025 parameter set to experimental voltage curves from the same cell. 

Let's import BattMo and some other packages we will use.

In [ ]:
using BattMo, GLMakie, CSV, DataFrames, Jutul

GLMakie.activate!(inline=true)

### Load the experimental data

We will calibrate our model in two steps: 
1. We will adjust the stoichiometric coefficients and maximum concentrations of the active materials, to fit a cell voltage curve at C/2.
2. We will adjust the reaction rate constants and diffusion coefficients in the active materials, to fit a cell voltage curve at 2C.

We first load the datasets.

In [ ]:
expdata_05C = CSV.read("data/Xu_2015_voltageCurve_05C.csv", DataFrame)
expdata_1C = CSV.read("data/Xu_2015_voltageCurve_1C.csv", DataFrame)
expdata_2C = CSV.read("data/Xu_2015_voltageCurve_2C.csv", DataFrame)

### Run a simulation of the original parameters
Now we run a baseline simulation using the parameters obtained only from characterization of the cell. We load the parameter set, ensure an appropiate lower voltage limit and DRate, and run the simulation as we saw in previous tutorials.

In [ ]:
cell_parameters_original = load_cell_parameters(; from_file_path = "default_sets/cell_parameters/Xu2015.json")
cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")

cycling_protocol["LowerVoltageLimit"] = 2.25
cycling_protocol["DRate"] = 0.5

model_setup = LithiumIonBattery()

sim_original = Simulation(model_setup, cell_parameters_original, cycling_protocol)
output_original = solve(sim_original);


Once the simulation completes, we can inspect the resutling voltage curves, and compare them with the experimental voltage curves.

In [ ]:
#Simulation data
time_series = output_original.time_series
simdata_time_original = time_series["Time"]
simdata_voltage_original = time_series["Voltage"]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5", xlabel = "Time / s", ylabel = "Voltage / V")
lines!(ax, simdata_time_original, simdata_voltage_original, label = "Simulation 0.5C: original parameters")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 20)
axislegend(position = :lb)
fig

We can see that the simulation with original parameters does not match well the experiment. Lets therefore fit some parameters to the experimental data.

# Low-rate Calibration
### Set up the low-rate calibration

We have developed a calibration function that takes as inputs the voltage and time arrays of the data, along with the initial simulation setup.

In [ ]:
calibration_low_rate = VoltageCalibration(expdata_05C[:,1], expdata_05C[:,2], sim_original); #  VoltageCalibration(experimental_time, experimental_voltage, simulation)

This calibration object is a handy way to tailor the main settings needed to run a calibration: 
* Which model parameters are frozen
* Which model parameters are being fitted
* What are the minimum and maximum bounds of the parameters to be fitted
* The results of the calibration, i.e. the optimal parameters.

**All paremters are frozen by default**, so we now need to free those we are interested in, and apply some bounds to each to ensure they remain within expected ranges. Below, we free the stoichiometric coefficients and maximum concentrations.

In [ ]:
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)

# "StoichiometricCoefficientAtSOC0" at both electrodes
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)

#  "MaximumConcentration" of both electrodes
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5);

We have a handy function to check parameter, values and bounds:

In [ ]:
print_calibration_overview(calibration_low_rate)

### Solve the low-rate calibration

Solving the calibration problem is essentially an optimization problem. We adjust free parameters so to minimize the difference between a target (the data) and the prediction (the simulation result): is performed by solving the optimization problem. This makes use of the adjoint method implemented in Jutul.jl and the LBFGS algorithm.

For calibration, we minimize the squared difference between the predicted and observed voltage, summed over all time steps:  
                  $\sum_i (V_i - V_{exp,i})^2$  
where $V_i$ is the voltage from the model and $V_{exp,i}$ is the voltage from the experimental data at step $i$. This minimization uses in the background cool algorithms implemented in Jutul, the simulation engine of BattMo. 

In [ ]:
solve(calibration_low_rate);
cell_parameters_calibrated_low_rate = calibration_low_rate.calibrated_cell_parameters;

We can use the same printing function to explore the results of the simulation

In [ ]:
print_calibration_overview(calibration_low_rate)

### Compare the results of the calibration against the experimental data

We can now use the optimized parameters to run a new simulation, and compare the results to the experimental data for the 0.5C discharge curve.

In [ ]:
#Setup and run simulation
sim_calibrated_low_rate = Simulation(model_setup, cell_parameters_calibrated_low_rate, cycling_protocol)
output_calibrated_low_rate = solve(sim_calibrated_low_rate);

#Get simulation data
time_series = output_calibrated_low_rate.time_series
simdata_time_calibrated_low_rate = time_series["Time"]
simdata_voltage_calibrated_low_rate = time_series["Voltage"]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5")
lines!(ax, simdata_time_original, simdata_voltage_original, label = "Simulation 0.5C: Original parameters")
lines!(ax, simdata_time_calibrated_low_rate, simdata_voltage_calibrated_low_rate, label = "Simulation 0.5C: after low rate calibration")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 20)
axislegend(position = :lb)
fig

# High-rate Calibration
### Set up the high-rate calibration

The second calibration is performed against the 2.0C discharge curve. In the same manner as for the first discharge curve, we set up a set of parameters to calibrate against experimental data. The parameters are:

 - The reaction rate constant of both electrodes
 - The diffusion coefficient of both electrodes

The calibration this time starts from the parameters calibrated at 0.5C, so we use the `cell_parameters_calibrated_05C` from the first `solve` to run a new simulation at 2C:

In [ ]:
#Update cycling protocol to run at 2C
cycling_protocol2 = deepcopy(cycling_protocol)
cycling_protocol2["DRate"] = 2.0

#Solve simulation with parameters calibrated at 05C but running a 2C discharge protocol
sim_calibrated_low_rate = Simulation(model_setup, cell_parameters_calibrated_low_rate, cycling_protocol2)
output_calibrated_low_rate = solve(sim_calibrated_low_rate);

#Get simulation data of parameters calibrated at 0.5C but run at 2C
time_series = output_calibrated_low_rate.time_series
simdata_time_calibrated_low_rate = time_series["Time"]
simdata_voltage_calibrated_low_rate = time_series["Voltage"]

We use the simulation at 2C ran with the parameter set calibrated at 0.5C as a starting point for our new high rate calibration task. 

This time we free the reaction rate constants and diffusion coefficients, and set some boundaries for each.

In [ ]:
calibration_high_rate = VoltageCalibration(expdata_2C[:,1], expdata_2C[:,2], sim_calibrated_low_rate)

free_calibration_parameter!(calibration_high_rate,
    ["NegativeElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)
free_calibration_parameter!(calibration_high_rate,
    ["PositiveElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)

free_calibration_parameter!(calibration_high_rate,
    ["NegativeElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)
free_calibration_parameter!(calibration_high_rate,
    ["PositiveElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)

print_calibration_overview(calibration_high_rate)

### Solve the high-rate calibration problem

In [ ]:
cell_parameters_calibrated_high_rate, = solve(calibration_high_rate);
print_calibration_overview(calibration_high_rate)

### Compare the results of the high-rate calibration against the experimental data

We compare three simulations against the experimental data:
 1. The initial simulation with the original parameters.
 2. The simulation with the parameters calibrated against the 0.5C discharge curve.
 3. The simulation with the parameters calibrated against the 0.5C and 2.0C discharge curves.

In [ ]:
# Simulation at 2C  using original parameters
sim_original_params = Simulation(model_setup, cell_parameters_original, cycling_protocol2)
output_original_params = solve(sim_original_params, accept_invalid = false);
 
time_series = output_original_params.time_series
simdata_time_original_params = time_series["Time"]
simdata_voltage_original_params = time_series["Voltage"]


# Simulation at 2C using calibrated parameters from 2C calibration
sim_calibrated_high_rate = Simulation(model_setup, cell_parameters_calibrated_high_rate, cycling_protocol2)
output_calibrated_high_rate = solve(sim_calibrated_high_rate, accept_invalid = false);
 
time_series = output_calibrated_high_rate.time_series
simdata_time_calibrated_high_rate = time_series["Time"]
simdata_voltage_calibrated_high_rate = time_series["Voltage"]

# Plot 2C calibrated model vs 2C experimental data
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 2.0")
lines!(ax, simdata_time_original_params, simdata_voltage_original_params, label = "Simulation 2C: Original parameters")
lines!(ax, simdata_time_calibrated_low_rate, simdata_voltage_calibrated_low_rate, label = "Simulation 2C: after low-rate calibration")
lines!(ax, simdata_time_calibrated_high_rate, simdata_voltage_calibrated_high_rate, label = "Simulation 2C: after high-rate calibration")
scatter!(ax, expdata_2C[:,1], expdata_2C[:,2], label = "Experimental data 2C", markersize = 20)
axislegend(position = :lb)
fig

# Calibrated model at all CRates

We can now compare the results of the model after both low-rate and high-rate calibration against the experimental data for the 0.5C, 1.0C, and 2.0C discharge curves. 

> **Note that we did not calibrate the model for the 1.0C discharge curve, but we still obtain a good fit!!**

In [ ]:
CRates = [0.5, 1.0, 2.0]
colors = Dict(0.5 => :firebrick1, 1.0 => :teal, 2.0 => :dodgerblue4)

fig = Figure()
ax = Axis(fig[1, 1], title = "Simulations vs. Experiments: after calibration")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 15, color = colors[0.5])
scatter!(ax, expdata_1C[:,1], expdata_1C[:,2], label = "Experimental data 1C", markersize = 15, color =colors[1.0])
scatter!(ax, expdata_2C[:,1], expdata_2C[:,2], label = "Experimental data 2C", markersize = 15, color =colors[2.0])

for CRate in CRates
    #Setup and run simulation
    cycling_protocol["DRate"] = CRate
    sim = Simulation(model_setup, cell_parameters_calibrated_high_rate, cycling_protocol)
    output = solve(sim, info_level = -1)

    #Get time series quantities from simulation result
    t = output.time_series["Time"]
    V = output.time_series["Voltage"]

    #Plot simulation voltage response
    lines!(ax, t, V, label = "Simulation $CRate: after high-rate calibration", color = colors[CRate])
end  

axislegend(position = :lb)
fig